In [ ]:
!pip install langchain openai wikipedia

In [50]:
from langchain.docstore.wikipedia import Wikipedia
from langchain.agents.react.base import DocstoreExplorer
from langchain.agents import Tool
from langchain.llms import OpenAI
from langchain.agents import initialize_agent
from langchain.callbacks import get_openai_callback

In [51]:
docstore = DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description='search wikipedia'
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description='lookup a term in wikipedia'
    )
]

In [53]:
llm = OpenAI(temperature=0)
docstore_agent=initialize_agent(
    tools,
    llm,
    agent="react-docstore",
    verbose=True,
    max_iterations=3
)

In [55]:
docstore_agent("What is the capitol of Japan")



> Entering new AgentExecutor chain...
Thought: I need to search Japan and find its capitol.
Action: Search[Japan]
Observation: Japan (Japanese: 日本, Nippon or Nihon, and formally 日本国, Nihonkoku) is an island country in East Asia. It is situated in the northwest Pacific Ocean and is bordered on the west by the Sea of Japan, extending from the Sea of Okhotsk in the north toward the East China Sea, Philippine Sea, and Taiwan in the south. Japan is a part of the Ring of Fire, and spans an archipelago of 14,125 islands, with the five main islands being Hokkaido, Honshu (the "mainland"), Shikoku, Kyushu, and Okinawa. Tokyo is the nation's capital and largest city, followed by Yokohama, Osaka, Nagoya, Sapporo, Fukuoka, Kobe, and Kyoto.
Japan is the eleventh most populous country in the world, as well as one of the most densely populated and urbanized. About three-fourths of the country's terrain is mountainous, concentrating its population of almost 125 million on narrow coastal plains. Japa

{'input': 'What is the capitol of Japan', 'output': 'Tokyo'}

In [18]:

def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
    return {
        'result': result,
        'token_count': cb.total_tokens
    }

In [19]:
count_tokens(docstore_agent, "What is Icarus known for?")



> Entering new AgentExecutor chain...
Thought: I need to search Icarus and find what it is known for.
Action: Search[Icarus]
Observation: In Greek mythology, Icarus (; Ancient Greek: Ἴκαρος, romanized: Íkaros, pronounced [ǐːkaros]) was the son of the master craftsman Daedalus, the architect of the labyrinth of Crete. After Theseus, king of Athens and enemy of Minos, escaped from the labyrinth, King Minos suspected that Icarus and Daedalus had revealed the labyrinth's secrets and imprisoned them—either in a large tower overlooking the ocean or the labyrinth itself, depending upon the account. Icarus and Daedalus escaped using wings Daedalus constructed from feathers, threads from blankets, clothes, and beeswax. Daedalus warned Icarus first of complacency and then of hubris, instructing him to fly neither too low nor too high, lest the sea's dampness clog his wings or the sun's heat melt them. Icarus ignored Daedalus’ instructions not to fly too close to the sun, causing the beeswax in

{'result': 'flying too close to the sun, causing the beeswax in his wings to melt and him to fall from the sky and drown',
 'token_count': 3329}

In [42]:
from langchain import OpenAI
from langchain.agents import initialize_agent, Tool
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate(
    input_variables=["query"],
    template="{query}"
)

llm_chain = LLMChain(llm=llm, prompt=prompt)


from langchain.agents import load_tools
search =GoogleSearchAPIWrapper()
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
        return_direct=True
    ),
     Tool(
       name='Language Model',
       func=llm_chain.run,
       description='use this tool for general purpose queries and logic'
    )
    
]

In [43]:
agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3
)

In [49]:
agent("What is the capitol of Japan")



> Entering new AgentExecutor chain...
 I need to find out what the capitol of Japan is
Action: Search
Action Input: "capitol of Japan"
Observation: Experience true relaxation in a peaceful setting appointed with sophisticated Japanese aesthetics, embracing the changing seasons. The current capital of Japan is Tokyo. In the course of history, the national capital has been in many locations other than Tokyo. Travis Japan. Spotify logo · Apple Music logo · YouTube logo · Instagram logo · Twitter logo. JUST DANCE! Listen Now. applemusic logo · spotify logo. SANTA MONICA and TOKYO – September 28, 2022 – Capitol Music Group (CMG), a division of Universal Music Group (UMG), the world leader in music-based ... Many other states and the nation's Capitol also boast Japanese blossoming cherry trees in the spring. In Japan, blossoming cherry trees, with their rapidly ... Capitol Hill Dialogue on Japan's Position in U.S.-China Relations. Aug 25, 2020. On Tuesday, August 25th, Mansfield Foundation

{'input': 'What is the capitol of Japan',
 'output': "Experience true relaxation in a peaceful setting appointed with sophisticated Japanese aesthetics, embracing the changing seasons. The current capital of Japan is Tokyo. In the course of history, the national capital has been in many locations other than Tokyo. Travis Japan. Spotify logo · Apple Music logo · YouTube logo · Instagram logo · Twitter logo. JUST DANCE! Listen Now. applemusic logo · spotify logo. SANTA MONICA and TOKYO – September 28, 2022 – Capitol Music Group (CMG), a division of Universal Music Group (UMG), the world leader in music-based\xa0... Many other states and the nation's Capitol also boast Japanese blossoming cherry trees in the spring. In Japan, blossoming cherry trees, with their rapidly\xa0... Capitol Hill Dialogue on Japan's Position in U.S.-China Relations. Aug 25, 2020. On Tuesday, August 25th, Mansfield Foundation President & CEO Frank Jannuzi\xa0... Accommodations. Each of the 251 guestrooms and suite

In [68]:
!pip install wolframalpha

In [71]:
from langchain.memory import ConversationBufferMemory


In [72]:
llm = OpenAI(temperature=0)
tools = load_tools(['wolfram-alpha'])
memory = ConversationBufferMemory(memory_key="chat_history")
conversational_agent = initialize_agent(tools, llm, agent="conversational-react-description", memory=memory, verbose=True)

In [73]:
conversational_agent("hi how are you")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
AI: Hi there! I'm doing great, how about you?

> Finished chain.


{'input': 'hi how are you',
 'chat_history': '',
 'output': "Hi there! I'm doing great, how about you?"}

In [74]:
conversational_agent("How far is it from Osaka to Tokyo")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Wolfram Alpha
Action Input: Distance from Osaka to Tokyo
Observation: Assumption: distance | from | Osaka, Japan
to | Tokyo, Japan 
Answer: 251.1 miles
Thought: Do I need to use a tool? No
AI: The distance from Osaka to Tokyo is 251.1 miles.

> Finished chain.


{'input': 'How far is it from Osaka to Tokyo',
 'chat_history': "Human: hi how are you\nAI: Hi there! I'm doing great, how about you?",
 'output': 'The distance from Osaka to Tokyo is 251.1 miles.'}

In [75]:
conversational_agent("What is the Tokio known for?")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Wolfram Alpha
Action Input: What is Tokyo known for?
Observation: Assumption: Tokyo, Japan 
Answer: city population | 13.99 million people (country rank: 1st) (2022 estimate)
metro area population | 37.2 million people (Tokyo metro area) (2007 estimate)
Thought: Do I need to use a tool? No
AI: Tokyo is known for its large population, being the most populous city in Japan and one of the most populous cities in the world. It is also known for its vibrant culture, with many attractions, restaurants, and shopping areas.

> Finished chain.


{'input': 'What is the Tokio known for?',
 'chat_history': "Human: hi how are you\nAI: Hi there! I'm doing great, how about you?\nHuman: How far is it from Osaka to Tokyo\nAI: The distance from Osaka to Tokyo is 251.1 miles.",
 'output': 'Tokyo is known for its large population, being the most populous city in Japan and one of the most populous cities in the world. It is also known for its vibrant culture, with many attractions, restaurants, and shopping areas.'}

In [76]:
conversational_agent("What would be the average cost of Sushi in the downtown area?")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Wolfram Alpha
Action Input: Average cost of sushi in Tokyo
Observation: Wolfram Alpha wasn't able to answer it
Thought: Do I need to use a tool? No
AI: The average cost of sushi in Tokyo's downtown area can vary depending on the type of sushi and the restaurant. Generally, you can expect to pay anywhere from ¥500 to ¥2,000 per plate.

> Finished chain.


{'input': 'What would be the average cost of Sushi in the downtown area?',
 'chat_history': "Human: hi how are you\nAI: Hi there! I'm doing great, how about you?\nHuman: How far is it from Osaka to Tokyo\nAI: The distance from Osaka to Tokyo is 251.1 miles.\nHuman: What is the Tokio known for?\nAI: Tokyo is known for its large population, being the most populous city in Japan and one of the most populous cities in the world. It is also known for its vibrant culture, with many attractions, restaurants, and shopping areas.",
 'output': "The average cost of sushi in Tokyo's downtown area can vary depending on the type of sushi and the restaurant. Generally, you can expect to pay anywhere from ¥500 to ¥2,000 per plate."}

In [78]:
tools.append(["google-search"])

In [79]:
conversational_agent("What's the weather like there now?")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Wolfram Alpha
Action Input: Weather in Tokyo
Observation: Assumption: weather | Tokyo 
Answer: temperature | 68 °F
relative humidity | 73% (dew point: 59 °F)
wind speed | 14 mph
(56 minutes ago)
Thought: Do I need to use a tool? No
AI: The current temperature in Tokyo is 68 °F, with a relative humidity of 73% (dew point: 59 °F) and a wind speed of 14 mph.

> Finished chain.


{'input': "What's the weather like there now?",
 'chat_history': "Human: hi how are you\nAI: Hi there! I'm doing great, how about you?\nHuman: How far is it from Osaka to Tokyo\nAI: The distance from Osaka to Tokyo is 251.1 miles.\nHuman: What is the Tokio known for?\nAI: Tokyo is known for its large population, being the most populous city in Japan and one of the most populous cities in the world. It is also known for its vibrant culture, with many attractions, restaurants, and shopping areas.\nHuman: What would be the average cost of Sushi in the downtown area?\nAI: The average cost of sushi in Tokyo's downtown area can vary depending on the type of sushi and the restaurant. Generally, you can expect to pay anywhere from ¥500 to ¥2,000 per plate.",
 'output': 'The current temperature in Tokyo is 68 °F, with a relative humidity of 73% (dew point: 59 °F) and a wind speed of 14 mph.'}